<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2021notebooks/2021_0702RNN_binary_addtion_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# リカレントニューラルネットワークによる足し算のデモ


In [ ]:
import numpy as np
import sys
import copy    # for deepcopy

In [ ]:
def sigmoid(x):
    return 1./(1.+np.exp(-x))

def d_sigmoid(x):
    return x * (1.-x)

int2binary = {}
binary_dim = 8

largest_number = pow(2,binary_dim)
binary = np.unpackbits(
    np.array([range(largest_number)], dtype=np.uint8).T, axis=1)

for i in range(largest_number):
    int2binary[i] = binary[i]

### hyperparameters
lr = 0.1
iter_max = 2 * 10 ** 4
interval = iter_max >> 3
n_inp = 2  #入力層ニューロン数
n_hid = 8  #中間層ニューロン数
n_out = 1  #出力層ニューロン数

W_ih = 2 * np.random.random((n_inp, n_hid)) - 1.
W_ho = 2 * np.random.random((n_hid, n_out)) - 1.
W_hh = 2 * np.random.random((n_hid, n_hid)) - 1.

dW_ih = np.zeros_like(W_ih)
dW_ho = np.zeros_like(W_ho)
dW_hh = np.zeros_like(W_hh)

In [ ]:
for i in range(iter_max):
    a_int = np.random.randint(largest_number / 2)
    a = int2binary[a_int]
    b_int = np.random.randint(largest_number / 2)
    b = int2binary[b_int]
    c_int = a_int + b_int
    c = int2binary[c_int]
    d = np.zeros_like(c)

    total_err = 0
    O_deltas = list()
    H_list = list()
    H_list.append(np.zeros(n_hid))

    # forward prop
    for pos in range(binary_dim):
        k = binary_dim - pos - 1
        X = np.array([[a[k], b[k]]])
        y = np.array([[c[k]]]).T

        H = sigmoid(np.dot(X, W_ih) + np.dot(H_list[-1], W_hh))
        O = sigmoid(np.dot(H, W_ho))

        Delta = y - O  #誤差の計算 
        O_deltas.append((Delta) * d_sigmoid(O))
        total_err += np.abs(Delta[0])
        
        d[k] = np.round(O[0][0])  #予測出力のために d に予測値を保存
        H_list.append(copy.deepcopy(H))  # 次の時刻の処理のために中間層の値を保存

    # back prop
    future_H_delta = np.zeros(n_hid)
    for pos in range(binary_dim):
        X        = np.array([[a[pos], b[pos]]])
        H      = H_list[- pos - 1]
        H_prev = H_list[- pos - 2]

        O_delta = O_deltas[- pos - 1]
        H_delta = (future_H_delta.dot(W_hh.T) + O_delta.dot(W_ho.T)) * d_sigmoid(H)

        dW_ho += np.atleast_2d(     H).T.dot(O_delta)
        dW_hh += np.atleast_2d(H_prev).T.dot(H_delta)
        dW_ih += X.T.dot(H_delta)

        future_H_delta = H_delta

    W_ho += lr * dW_ho
    W_hh += lr * dW_hh
    W_ih += lr * dW_ih

    dW_ho = 0
    dW_hh = 0
    dW_ih = 0

    if i % interval == 0:
        out = 0
        for index, x in enumerate(reversed(d)):
            out += x * pow(2,index)
        print(f'{i:>5d} 誤差: {total_err[0]:.3f}', end=': ')
        print(f'{str(a_int)} + {str(b_int)} = {str(out)}')
        print(f'予測: {str(d)}')
        print(f'正解: {str(c)}')